## Initialize OpenAI Client
Set up the OpenAI client with Nebius AI's API endpoint and your API key

In [ ]:
import os
from openai import OpenAI
import time

client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

## Upload Training and Validation Datasets
Nebius AI requires datasets in `.jsonl` format. Upload your training and validation datasets.

In [ ]:
# Upload a training dataset
training_dataset = client.files.create(
    file=open("<dataset_name>.jsonl", "rb"), # Specify the dataset name
    purpose="fine-tune"
)

# Upload a validation dataset
validation_dataset = client.files.create(
    file=open("<dataset_name>.jsonl", "rb"), # Specify the dataset name
    purpose="fine-tune"
)

## Create Fine-Tuning Job
Specify the model and fine-tuning parameters, including number of epochs and LoRA adaptation.

If you want to add more Parameters, Check your [Nebius Docs](https://docs.nebius.com/studio/fine-tuning/how-to-fine-tune#instructions) for more information.

In [ ]:
job_request = {
    "model": "meta-llama/Llama-3.1-8B-Instruct", # Choose the model
    "training_file": training_dataset.id,
    "validation_file": validation_dataset.id,
    "hyperparameters": {
        "n_epochs": 3, # Number of epochs for training
        "lora": True,  # Enable LoRA for fine-tuning efficiency
    },
    "integrations": [],
}

# Create and run the fine-tuning job
job = client.fine_tuning.jobs.create(**job_request)

## Monitor Fine-Tuning Status
Check the job status periodically to determine whether it is running, completed, or failed.

In [ ]:
# Create and run the fine-tuning job
job = client.fine_tuning.jobs.create(**job_request)

# Make sure that the job has been finished or cancelled
active_statuses = ["validating_files", "queued", "running"]
while job.status in active_statuses:
    time.sleep(15)
    job = client.fine_tuning.jobs.retrieve(job.id)
    print("current status is", job.status)

print("Job ID:", job.id)

## Retrieve Fine-Tuned Model Checkpoints
If the fine-tuning job succeeds, retrieve the trained model checkpoints.

In [ ]:
if job.status == "succeeded":
    # Check the job events
    events = client.fine_tuning.jobs.list_events(job.id)
    print(events)

    for checkpoint in client.fine_tuning.jobs.checkpoints.list(job.id).data:
        print("Checkpoint ID:", checkpoint.id)

        # Create a directory for every checkpoint
        os.makedirs(checkpoint.id, exist_ok=True)

        for model_file_id in checkpoint.result_files:
            # Get the name of a model file
            filename = client.files.retrieve(model_file_id).filename

            # Retrieve the contents of the file
            file_content = client.files.content(model_file_id)

            # Save the contents into a local file
            file_content.write_to_file(filename)